# Prepare

### Project Requirements

- Data Prep: Column data types are appropriate for the data they contain
- Data Prep: Missing values are investigated and handled
- Data Prep: Outliers are investigated and handled

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from env import get_db_url
import acquire
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = acquire.get_zillow_data()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77380 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49671 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77184 non-null  float64
 13  f

In [4]:
# sets thresh hold to 50 percent nulls
threshold = df.shape[0] * .50

# remove columns with high nulls
df = df.dropna(axis=1, thresh=threshold)

df.columns

Index(['id', 'parcelid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fips', 'fullbathcnt', 'heatingorsystemtypeid',
       'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidzip', 'roomcnt', 'unitcnt', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock', 'logerror',
       'transactiondate', 'heatingorsystemdesc', 'propertylandusedesc'],
      dtype='object')

In [5]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52319
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

#### should only need to analyze single family residental

In [6]:
# only list single family residental
df = df [(df.propertylandusedesc == 'Single Family Residential')]

In [7]:
df.bedroomcnt.value_counts()

3.0     23302
4.0     15210
2.0      8317
5.0      3967
6.0       634
1.0       611
0.0       135
7.0       106
8.0        24
9.0         8
10.0        2
14.0        1
11.0        1
12.0        1
Name: bedroomcnt, dtype: int64

In [8]:
df.bathroomcnt.value_counts()

2.0     21839
3.0     10650
1.0      9536
2.5      3932
4.0      2225
3.5       916
1.5       839
5.0       803
4.5       686
6.0       320
5.5       224
0.0       119
7.0        88
8.0        53
6.5        47
7.5        16
9.0        13
10.0        5
11.0        3
8.5         3
18.0        1
13.0        1
Name: bathroomcnt, dtype: int64

In [9]:
# lets only anaylze >0 and <6
df = df[(df.bedroomcnt <= 6 ) & (df.bathroomcnt <= 6 ) & (df.bedroomcnt > 0 ) & (df.bathroomcnt > 0) ]

In [10]:
df.fips.value_counts()

6037.0    33534
6059.0    13972
6111.0     4335
Name: fips, dtype: int64

In [11]:
df.bedroomcnt.value_counts()

3.0    23294
4.0    15189
2.0     8315
5.0     3891
1.0      607
6.0      545
Name: bedroomcnt, dtype: int64

In [12]:
df.bathroomcnt.value_counts()

2.0    21827
3.0    10621
1.0     9518
2.5     3930
4.0     2202
3.5      913
1.5      839
5.0      787
4.5      679
6.0      302
5.5      223
Name: bathroomcnt, dtype: int64

In [13]:
# keep only properties less than 3000 square feet. same as regression project.
df = df[df.calculatedfinishedsquarefeet <= 3000 ]    

In [14]:
df.fips.value_counts()

6037.0    30490
6059.0    11934
6111.0     3706
Name: fips, dtype: int64

In [15]:
# keep only properties less than 1m.
df = df[df.taxvaluedollarcnt <= 1000000]  

In [16]:
df.fips.value_counts()

6037.0    28834
6059.0    11379
6111.0     3652
Name: fips, dtype: int64

In [17]:
df.unitcnt.value_counts()

1.0    28793
2.0       14
3.0        1
Name: unitcnt, dtype: int64

In [18]:
df.roomcnt.value_counts()
#same as bedrooms. will drop

0.0     31122
6.0      3809
7.0      3369
8.0      2421
5.0      1794
9.0       701
4.0       474
10.0       91
3.0        63
11.0       10
2.0         9
12.0        1
1.0         1
Name: roomcnt, dtype: int64

In [19]:
df.calculatedbathnbr.value_counts()
# already showed in bathrooms as a float

2.0    20976
1.0     9333
3.0     8254
2.5     3626
1.5      823
4.0      545
3.5      240
5.0       30
4.5       21
6.0        3
5.5        2
Name: calculatedbathnbr, dtype: int64

In [20]:
df.propertylandusedesc.value_counts()

Single Family Residential    43865
Name: propertylandusedesc, dtype: int64

In [21]:
df.fips.value_counts()

6037.0    28834
6059.0    11379
6111.0     3652
Name: fips, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43865 entries, 1 to 77379
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            43865 non-null  int64  
 1   parcelid                      43865 non-null  int64  
 2   bathroomcnt                   43865 non-null  float64
 3   bedroomcnt                    43865 non-null  float64
 4   buildingqualitytypeid         28808 non-null  float64
 5   calculatedbathnbr             43853 non-null  float64
 6   calculatedfinishedsquarefeet  43865 non-null  float64
 7   finishedsquarefeet12          43745 non-null  float64
 8   fips                          43865 non-null  float64
 9   fullbathcnt                   43853 non-null  float64
 10  heatingorsystemtypeid         28872 non-null  float64
 11  latitude                      43865 non-null  float64
 12  longitude                     43865 non-null  float64
 13  l

In [23]:
df.buildingqualitytypeid.value_counts(), df.buildingqualitytypeid.mean()

(6.0     9904
 4.0     7999
 8.0     7300
 7.0     1752
 5.0     1487
 9.0      191
 3.0      147
 11.0      20
 2.0        3
 1.0        3
 10.0       2
 Name: buildingqualitytypeid, dtype: int64,
 5.968064426548181)

In [24]:
df.isnull().sum()

id                                  0
parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           15057
calculatedbathnbr                  12
calculatedfinishedsquarefeet        0
finishedsquarefeet12              120
fips                                0
fullbathcnt                        12
heatingorsystemtypeid           14993
latitude                            0
longitude                           0
lotsizesquarefeet                 217
propertycountylandusecode           0
propertylandusetypeid               0
propertyzoningdesc              15068
rawcensustractandblock              0
regionidcity                      732
regionidcounty                      0
regionidzip                         9
roomcnt                             0
unitcnt                         15057
yearbuilt                          21
structuretaxvaluedollarcnt         28
taxvaluedollarcnt                   0
assessmentye

In [25]:
#df = df.dropna()
df = df.fillna(df.mean())

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43865 entries, 1 to 77379
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            43865 non-null  int64  
 1   parcelid                      43865 non-null  int64  
 2   bathroomcnt                   43865 non-null  float64
 3   bedroomcnt                    43865 non-null  float64
 4   buildingqualitytypeid         43865 non-null  float64
 5   calculatedbathnbr             43865 non-null  float64
 6   calculatedfinishedsquarefeet  43865 non-null  float64
 7   finishedsquarefeet12          43865 non-null  float64
 8   fips                          43865 non-null  float64
 9   fullbathcnt                   43865 non-null  float64
 10  heatingorsystemtypeid         43865 non-null  float64
 11  latitude                      43865 non-null  float64
 12  longitude                     43865 non-null  float64
 13  l

In [27]:
df.fips.value_counts()

6037.0    28834
6059.0    11379
6111.0     3652
Name: fips, dtype: int64

In [28]:
df

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
1,1387261,17052889,1.0,2.0,5.968064,1.0,1465.0,1465.0,6111.0,1.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,NaN,Single Family Residential
2,11677,14186244,2.0,3.0,5.968064,2.0,1243.0,1243.0,6059.0,2.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,NaN,Single Family Residential
3,2288172,12177905,3.0,4.0,8.000000,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
6,781532,12095076,3.0,4.0,9.000000,3.0,2962.0,2962.0,6037.0,3.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential
7,870991,12069064,1.0,2.0,5.000000,1.0,738.0,738.0,6037.0,1.0,...,18890.0,218552.0,2016.0,199662.0,2366.08,6.037302e+13,0.101723,2017-01-01,NaN,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77374,2274245,12412492,2.0,4.0,6.000000,2.0,1633.0,1633.0,6037.0,2.0,...,125466.0,346534.0,2016.0,221068.0,4175.08,6.037555e+13,0.001082,2017-09-19,Central,Single Family Residential
77376,673515,11000655,2.0,2.0,6.000000,2.0,1286.0,1286.0,6037.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential
77377,2968375,17239384,2.0,4.0,5.968064,2.0,1612.0,1612.0,6111.0,2.0,...,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,NaN,Single Family Residential
77378,1843709,12773139,1.0,3.0,4.000000,1.0,1032.0,1032.0,6037.0,1.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential


In [29]:
df.shape

(43865, 34)

In [30]:
df =df.drop(columns= ['finishedsquarefeet12', 'fullbathcnt', 'calculatedbathnbr',
                      'propertyzoningdesc', 'unitcnt', 'propertylandusedesc',
                      'assessmentyear', 'roomcnt', 'regionidcounty', 'propertylandusetypeid',
                      'heatingorsystemtypeid', 'id', 'heatingorsystemdesc', 'buildingqualitytypeid'],axis=1)

In [31]:
df.fips.value_counts()

6037.0    28834
6059.0    11379
6111.0     3652
Name: fips, dtype: int64

In [32]:
df.shape

(43865, 20)

In [33]:
df.columns

Index(['parcelid', 'bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet',
       'fips', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'rawcensustractandblock', 'regionidcity',
       'regionidzip', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount',
       'censustractandblock', 'logerror', 'transactiondate'],
      dtype='object')

In [34]:
def overview(df):
    print('--- Shape: {}'.format(df.shape))
    print('--- Info')
    df.info()
    print('--- Column Descriptions')
    print(df.describe(include='all'))
    return df

In [35]:
overview(df)

--- Shape: (43865, 20)
--- Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 43865 entries, 1 to 77379
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      43865 non-null  int64  
 1   bathroomcnt                   43865 non-null  float64
 2   bedroomcnt                    43865 non-null  float64
 3   calculatedfinishedsquarefeet  43865 non-null  float64
 4   fips                          43865 non-null  float64
 5   latitude                      43865 non-null  float64
 6   longitude                     43865 non-null  float64
 7   lotsizesquarefeet             43865 non-null  float64
 8   propertycountylandusecode     43865 non-null  object 
 9   rawcensustractandblock        43865 non-null  float64
 10  regionidcity                  43865 non-null  float64
 11  regionidzip                   43865 non-null  float64
 12  yearbuilt                   

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,regionidcity,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1110,6.111001e+07,13091.0,97099.0,1967.0,88000.0,464000.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,122,6.059022e+07,21412.0,97078.0,1962.0,85289.0,564778.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,0101,6.037300e+07,396551.0,96330.0,1970.0,108918.0,145143.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,0101,6.037461e+07,47019.0,96293.0,1950.0,276684.0,773303.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01
7,12069064,1.0,2.0,738.0,6037.0,34149214.0,-118239357.0,4214.0,0100,6.037302e+07,45457.0,96325.0,1922.0,18890.0,218552.0,199662.0,2366.08,6.037302e+13,0.101723,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77374,12412492,2.0,4.0,1633.0,6037.0,33870815.0,-118070858.0,4630.0,0100,6.037555e+07,30267.0,96204.0,1962.0,125466.0,346534.0,221068.0,4175.08,6.037555e+13,0.001082,2017-09-19
77376,11000655,2.0,2.0,1286.0,6037.0,34245368.0,-118282383.0,47405.0,0100,6.037101e+07,12447.0,96284.0,1940.0,70917.0,354621.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20
77377,17239384,2.0,4.0,1612.0,6111.0,34300140.0,-118706327.0,12105.0,1111,6.111008e+07,27110.0,97116.0,1964.0,50683.0,67205.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21
77378,12773139,1.0,3.0,1032.0,6037.0,34040895.0,-118038169.0,5074.0,0100,6.037434e+07,36502.0,96480.0,1954.0,32797.0,49546.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21


In [36]:
#adding same features from regression project
df['transactiondate'] = df.transactiondate.astype('str')
df['transaction_month'] = df.transactiondate.str.split('-',expand=True)[1]

In [37]:
df.fips.value_counts()

6037.0    28834
6059.0    11379
6111.0     3652
Name: fips, dtype: int64

In [38]:
df

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,...,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,transaction_month
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1110,6.111001e+07,...,97099.0,1967.0,88000.0,464000.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,122,6.059022e+07,...,97078.0,1962.0,85289.0,564778.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,0101,6.037300e+07,...,96330.0,1970.0,108918.0,145143.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,01
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,0101,6.037461e+07,...,96293.0,1950.0,276684.0,773303.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,01
7,12069064,1.0,2.0,738.0,6037.0,34149214.0,-118239357.0,4214.0,0100,6.037302e+07,...,96325.0,1922.0,18890.0,218552.0,199662.0,2366.08,6.037302e+13,0.101723,2017-01-01,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77374,12412492,2.0,4.0,1633.0,6037.0,33870815.0,-118070858.0,4630.0,0100,6.037555e+07,...,96204.0,1962.0,125466.0,346534.0,221068.0,4175.08,6.037555e+13,0.001082,2017-09-19,09
77376,11000655,2.0,2.0,1286.0,6037.0,34245368.0,-118282383.0,47405.0,0100,6.037101e+07,...,96284.0,1940.0,70917.0,354621.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,09
77377,17239384,2.0,4.0,1612.0,6111.0,34300140.0,-118706327.0,12105.0,1111,6.111008e+07,...,97116.0,1964.0,50683.0,67205.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,09
77378,12773139,1.0,3.0,1032.0,6037.0,34040895.0,-118038169.0,5074.0,0100,6.037434e+07,...,96480.0,1954.0,32797.0,49546.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,09


In [39]:
df['tax_rate'] = (df.taxamount/df.taxvaluedollarcnt) * 100

In [40]:
df.fips.value_counts()

6037.0    28834
6059.0    11379
6111.0     3652
Name: fips, dtype: int64

In [41]:
df

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,transaction_month,tax_rate
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1110,6.111001e+07,...,1967.0,88000.0,464000.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,01,1.222517
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,122,6.059022e+07,...,1962.0,85289.0,564778.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,01,1.148823
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,0101,6.037300e+07,...,1970.0,108918.0,145143.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,01,1.224661
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,0101,6.037461e+07,...,1950.0,276684.0,773303.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,01,1.230599
7,12069064,1.0,2.0,738.0,6037.0,34149214.0,-118239357.0,4214.0,0100,6.037302e+07,...,1922.0,18890.0,218552.0,199662.0,2366.08,6.037302e+13,0.101723,2017-01-01,01,1.082616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77374,12412492,2.0,4.0,1633.0,6037.0,33870815.0,-118070858.0,4630.0,0100,6.037555e+07,...,1962.0,125466.0,346534.0,221068.0,4175.08,6.037555e+13,0.001082,2017-09-19,09,1.204811
77376,11000655,2.0,2.0,1286.0,6037.0,34245368.0,-118282383.0,47405.0,0100,6.037101e+07,...,1940.0,70917.0,354621.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,09,1.262878
77377,17239384,2.0,4.0,1612.0,6111.0,34300140.0,-118706327.0,12105.0,1111,6.111008e+07,...,1964.0,50683.0,67205.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,09,1.647913
77378,12773139,1.0,3.0,1032.0,6037.0,34040895.0,-118038169.0,5074.0,0100,6.037434e+07,...,1954.0,32797.0,49546.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,09,1.768922


In [42]:
    df['yearbuilt'] = df['yearbuilt'].astype(int)
    df["bedroomcnt"] = df["bedroomcnt"].astype(int)
    df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
    df["fips"] = df["fips"].astype(int)
    df["lotsizesquarefeet"] = df["lotsizesquarefeet"].astype(int)
    df["rawcensustractandblock"] = df["rawcensustractandblock"].astype(int)
    df["regionidcity"] = df["regionidcity"].astype(int)
    df["regionidzip"] = df["regionidzip"].astype(int)
    df["censustractandblock"] = df["censustractandblock"].astype(int)
    df["structuretaxvaluedollarcnt"] = df["structuretaxvaluedollarcnt"].astype(int)
    df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
    df["landtaxvaluedollarcnt"] = df["landtaxvaluedollarcnt"].astype(int)
    df["taxamount"] = df["taxamount"].astype(int)
    df.yearbuilt = df.yearbuilt.astype(object) 
    df['age'] = 2017-df['yearbuilt']
    df = df.drop(columns='yearbuilt')
    df['age'] = df['age'].astype('int')

In [43]:
overview(df)

--- Shape: (43865, 22)
--- Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 43865 entries, 1 to 77379
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      43865 non-null  int64  
 1   bathroomcnt                   43865 non-null  float64
 2   bedroomcnt                    43865 non-null  int64  
 3   calculatedfinishedsquarefeet  43865 non-null  int64  
 4   fips                          43865 non-null  int64  
 5   latitude                      43865 non-null  float64
 6   longitude                     43865 non-null  float64
 7   lotsizesquarefeet             43865 non-null  int64  
 8   propertycountylandusecode     43865 non-null  object 
 9   rawcensustractandblock        43865 non-null  int64  
 10  regionidcity                  43865 non-null  int64  
 11  regionidzip                   43865 non-null  int64  
 12  structuretaxvaluedollarcnt  

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,transaction_month,tax_rate,age
1,17052889,1.0,2,1465,6111,34449266.0,-119281531.0,12647,1110,61110010,...,88000,464000,376000,5672,61110010023006,0.055619,2017-01-01,01,1.222517,50
2,14186244,2.0,3,1243,6059,33886168.0,-117823170.0,8432,122,60590218,...,85289,564778,479489,6488,60590218022012,0.005383,2017-01-01,01,1.148823,55
3,12177905,3.0,4,2376,6037,34245180.0,-118240722.0,13038,0101,60373001,...,108918,145143,36225,1777,60373001001006,-0.103410,2017-01-01,01,1.224661,47
6,12095076,3.0,4,2962,6037,34145202.0,-118179824.0,63000,0101,60374608,...,276684,773303,496619,9516,60374608001014,-0.001011,2017-01-01,01,1.230599,67
7,12069064,1.0,2,738,6037,34149214.0,-118239357.0,4214,0100,60373020,...,18890,218552,199662,2366,60373020041001,0.101723,2017-01-01,01,1.082616,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77374,12412492,2.0,4,1633,6037,33870815.0,-118070858.0,4630,0100,60375548,...,125466,346534,221068,4175,60375548022001,0.001082,2017-09-19,09,1.204811,55
77376,11000655,2.0,2,1286,6037,34245368.0,-118282383.0,47405,0100,60371014,...,70917,354621,283704,4478,60371014003002,0.020615,2017-09-20,09,1.262878,77
77377,17239384,2.0,4,1612,6111,34300140.0,-118706327.0,12105,1111,61110084,...,50683,67205,16522,1107,61110084022016,0.013209,2017-09-21,09,1.647913,53
77378,12773139,1.0,3,1032,6037,34040895.0,-118038169.0,5074,0100,60374338,...,32797,49546,16749,876,60374338022005,0.037129,2017-09-21,09,1.768922,63


In [44]:
df.isnull().sum()

parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                    0
regionidzip                     0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
logerror                        0
transactiondate                 0
transaction_month               0
tax_rate                        0
age                             0
dtype: int64

In [45]:
df.fips.value_counts()

6037    28834
6059    11379
6111     3652
Name: fips, dtype: int64

In [46]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df

add_upper_outlier_columns(df, k=1.5)

df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,...,regionidcity_outliers,regionidzip_outliers,structuretaxvaluedollarcnt_outliers,taxvaluedollarcnt_outliers,landtaxvaluedollarcnt_outliers,taxamount_outliers,censustractandblock_outliers,logerror_outliers,tax_rate_outliers,age_outliers
1,17052889,1.0,2,1465,6111,34449266.0,-119281531.0,12647,1110,61110010,...,0.0,0.0,0.0,0.0,0.0,0.0,1.949823e+11,0.0,0.0,0.0
2,14186244,2.0,3,1243,6059,33886168.0,-117823170.0,8432,122,60590218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
3,12177905,3.0,4,2376,6037,34245180.0,-118240722.0,13038,0101,60373001,...,314654.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
6,12095076,3.0,4,2962,6037,34145202.0,-118179824.0,63000,0101,60374608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
7,12069064,1.0,2,738,6037,34149214.0,-118239357.0,4214,0100,60373020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0


In [47]:
outlier_cols = [col for col in df if col.endswith('_outliers')]
for col in outlier_cols:
    print('~~~\n' + col)
    data = df[col][df[col] > 0]
    print(data.describe())

~~~
parcelid_outliers
count    1.300000e+01
mean     1.414262e+08
std      2.407460e+07
min      6.163772e+07
25%      1.449578e+08
50%      1.496337e+08
75%      1.496340e+08
max      1.496352e+08
Name: parcelid_outliers, dtype: float64
~~~
bathroomcnt_outliers
count    841.000000
mean       0.666171
std        0.354965
min        0.250000
25%        0.250000
50%        0.750000
75%        0.750000
max        2.750000
Name: bathroomcnt_outliers, dtype: float64
~~~
bedroomcnt_outliers
count    181.0
mean       0.5
std        0.0
min        0.5
25%        0.5
50%        0.5
75%        0.5
max        0.5
Name: bedroomcnt_outliers, dtype: float64
~~~
calculatedfinishedsquarefeet_outliers
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: calculatedfinishedsquarefeet_outliers, dtype: float64
~~~
fips_outliers
count    3652.0
mean       19.0
std         0.0
min        19.0
25%        19.0
50%        19.0
75%        19.0
max        1

In [48]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe and return that dataframe'''  
    for col in col_list:
        # get quartiles
        q1, q3 = df[f'{col}'].quantile([.25, .75])  
        # calculate interquartile range
        iqr = q3 - q1   
        # get upper bound
        upper_bound = q3 + k * iqr 
        # get lower bound
        lower_bound = q1 - k * iqr   
        # return dataframe without outliers        
        df = df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]        
    return df

In [49]:
#removing outliers
df = remove_outliers(df, 3, ['lotsizesquarefeet', 'structuretaxvaluedollarcnt','rawcensustractandblock']) 

# Put all together in prepare.py

In [50]:
def prep_zillow(df):
    '''
    Returns clean dataframe
    '''
    df = data_prep(df)    
    df = df [(df.propertylandusedesc == 'Single Family Residential')]
    # Only show properties less than and equal to 6 bed/baths 
    df = df[(df.bedroomcnt <= 6 ) & (df.bathroomcnt <= 6 )]    
    # Remove properties where there are no baths and no beds
    df = df[df.bathroomcnt > 0]  
    df = df[df.bedroomcnt > 0]  
    # keep only properties less than 3000 square feet
    df = df[df.calculatedfinishedsquarefeet <= 3000 ]    
    # keep only properties less than 1m.
    df = df[df.taxvaluedollarcnt <= 1000000]  
    # Remove 13 rows where unit count is 2. The NaN's can be safely
    # assumed as 1 and were just mislabeled in other counties.  
    #df = df[df['unitcnt'] != 2]
    #df['unitcnt'].fillna(1) 
    # do not need any of finishedsquarefeet columns
    # removing these columns that are repeated and unnecessary
    df =df.drop(columns= ['finishedsquarefeet12', 'fullbathcnt', 'calculatedbathnbr',
                      'propertyzoningdesc', 'unitcnt', 'propertylandusedesc',
                      'assessmentyear', 'roomcnt', 'regionidcounty', 'propertylandusetypeid',
                      'heatingorsystemtypeid', 'id', 'heatingorsystemdesc', 'buildingqualitytypeid'],
            axis=1)    
    # The last nulls can be dropped altogether. 
    df = df.fillna(df.mean())
    # convert the following to int.
    df['yearbuilt'] = df['yearbuilt'].astype(int)
    df["bedroomcnt"] = df["bedroomcnt"].astype(int)
    df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
    df["fips"] = df["fips"].astype(int)
    df["lotsizesquarefeet"] = df["lotsizesquarefeet"].astype(int)
    df["rawcensustractandblock"] = df["rawcensustractandblock"].astype(int)
    df["regionidcity"] = df["regionidcity"].astype(int)
    df["regionidzip"] = df["regionidzip"].astype(int)
    df["censustractandblock"] = df["censustractandblock"].astype(int)
    df["structuretaxvaluedollarcnt"] = df["structuretaxvaluedollarcnt"].astype(int)
    df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
    df["landtaxvaluedollarcnt"] = df["landtaxvaluedollarcnt"].astype(int)
    df["taxamount"] = df["taxamount"].astype(int)
    df['tax_rate'] = (df.taxamount/df.taxvaluedollarcnt) * 100
    df.yearbuilt = df.yearbuilt.astype(object) 
    df['age'] = 2017-df['yearbuilt']
    df = df.drop(columns='yearbuilt')
    df['age'] = df['age'].astype('int')
    df['age_bin'] = pd.cut(df.age, 
                           bins = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140],
                           labels = [0, .066, .133, .20, .266, .333, .40, .466, .533, 
                                     .60, .666, .733, .8, .866, .933])

    # add month feature
    df['transactiondate'] = df.transactiondate.astype('str')
    df['transaction_month'] = df.transactiondate.str.split('-',expand=True)[1]
    # add county names for fips feature
    df['county'] = np.where(df.fips == 6037, 'Los Angeles', np.where(df.fips == 6059, 'Orange','Ventura') )
    #df = df.drop(columns = ‘fips’)
    print('yearbuilt converted to age, added month of transaction, added County names. \n')   
    # Removing outliers 
    df = remove_outliers(df, 3, ['lotsizesquarefeet', 'structuretaxvaluedollarcnt','rawcensustractandblock']) 


    return df

In [51]:
df

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,...,regionidcity_outliers,regionidzip_outliers,structuretaxvaluedollarcnt_outliers,taxvaluedollarcnt_outliers,landtaxvaluedollarcnt_outliers,taxamount_outliers,censustractandblock_outliers,logerror_outliers,tax_rate_outliers,age_outliers
1,17052889,1.0,2,1465,6111,34449266.0,-119281531.0,12647,1110,61110010,...,0.0,0.0,0.0,0.0,0.0,0.0,1.949823e+11,0.0,0.000000,0.0
2,14186244,2.0,3,1243,6059,33886168.0,-117823170.0,8432,122,60590218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0
3,12177905,3.0,4,2376,6037,34245180.0,-118240722.0,13038,0101,60373001,...,314654.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0
7,12069064,1.0,2,738,6037,34149214.0,-118239357.0,4214,0100,60373020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0
10,11104527,3.0,4,2540,6037,34434971.0,-118464390.0,10384,0101,60379200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77373,10722691,2.0,3,1570,6037,34194943.0,-118629218.0,7499,0100,60371352,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0
77374,12412492,2.0,4,1633,6037,33870815.0,-118070858.0,4630,0100,60375548,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0
77377,17239384,2.0,4,1612,6111,34300140.0,-118706327.0,12105,1111,61110084,...,0.0,0.0,0.0,0.0,0.0,0.0,1.950563e+11,0.0,0.000000,0.0
77378,12773139,1.0,3,1032,6037,34040895.0,-118038169.0,5074,0100,60374338,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.042679,0.0


In [52]:
df.fips.value_counts()

6037    27152
6059    11137
6111     3381
Name: fips, dtype: int64